In [1]:
#Importacion de librerias
import bempp.api
import numpy as np 
import time
from readoff import *
from readpqr import *
start = time.time()

In [2]:
#Datos Principales. 
em = 4.     #[-] Permitividad electrica interior.
ei = 10.    #[-] Permitividad electrica de la capa intermedia.
es = 80.    #[-] Permitividad electrica exterior.
k = 0.125   #[1/A] Inverso de la longitud de Debey-Huckel del fluido.

#Eleccion de resolver la ecuacion matricial en BEM/BEM
SF = True  #Strong form: True utiliza el precondicionador Mass Matrix. False no se utiliza ningun precondicionador.

#Ensablaje de los operadores de frontera: #'fmm' para moleculas con un gran mayor numero de vertices.
#'default_nonlocal' para moleculas con un pequeño numero de vertices.
Assemble = 'default_nonlocal' 

In [3]:
#Datos de la posicion en el espacio, carga y radios de los atomos de la molecula.
PC,Q,R = readpqr('PQR/Sphere5Q3.pqr')  #Eleccion del "pqr" de la molecula

In [4]:
#Generar las malla superficiales de la molecula
#En caso que la malla tenga huecos pequeños, con trimesh se obtiene la informacion de la malla original sin los huecos.
import trimesh
V1,F1 = read_off('Mallas_S/Sphere5R0.off') #Eleccion del nombre de la malla inferior en formato "off"
meshSP = trimesh.Trimesh(vertices = V1, faces= F1) 
mesh_split = meshSP.split()
print("Found %i meshes"%len(mesh_split))

vertices_split1 = mesh_split[0].vertices 
faces_split1 = mesh_split[0].faces   
grid1 = bempp.api.grid.grid.Grid(vertices_split1.transpose(), faces_split1.transpose()) #Creacion de la malla superficial interior

verts2,faces2 =read_off('Mallas_S/Sphere6R0.off') #Eleccion del nombre de la malla superior en formato "off"
grid2 = bempp.api.grid.grid.Grid(verts2.transpose(), faces2.transpose()) #Creacion de la malla superficial exterior

Found 1 meshes


In [5]:
#Generar espacios de funcionales del potencial y su derivada para el dominio Ωm y Ωi
dirichl_space1 = bempp.api.function_space(grid1, "P", 1)  #Potencial electrostatico en la interfaz interior.
neumann_space1 = bempp.api.function_space(grid1, "DP", 0) #Derivada del potencial electrostatico en la interfaz interior.
dirichl_space2 = bempp.api.function_space(grid2, "P", 1)  #Potencial electrostatico en la interfaz superior.
neumann_space2 = bempp.api.function_space(grid2, "DP", 0) #Derivada del potencial electrostatico en la interfaz superior.

print("DS1 dofs: {0}".format(dirichl_space1.global_dof_count))
print("NS1 dofs: {0}".format(neumann_space1.global_dof_count))
print("DS2 dofs: {0}".format(dirichl_space2.global_dof_count))
print("NS2 dofs: {0}".format(neumann_space2.global_dof_count))

DS1 dofs: 290
NS1 dofs: 576
DS2 dofs: 417
NS2 dofs: 830


In [6]:
#Generar los operadores de frontera
bempp.api.GLOBAL_PARAMETERS.fmm.expansion_order = 5
#Operadores de identidad
I1d = bempp.api.operators.boundary.sparse.identity(dirichl_space1, dirichl_space1, dirichl_space1) # 1
I1n = bempp.api.operators.boundary.sparse.identity(dirichl_space1, neumann_space1, neumann_space1) # 1
I2d = bempp.api.operators.boundary.sparse.identity(dirichl_space2, dirichl_space2, dirichl_space2) # 1
I2n = bempp.api.operators.boundary.sparse.identity(dirichl_space2, neumann_space2, neumann_space2) # 1

#Dominio del soluto Ωm
K111 = bempp.api.operators.boundary.laplace.double_layer(dirichl_space1, dirichl_space1, dirichl_space1, assembler=Assemble) #K
V111 = bempp.api.operators.boundary.laplace.single_layer(neumann_space1, dirichl_space1, dirichl_space1, assembler=Assemble) #V
Y121 = bempp.api.ZeroBoundaryOperator(dirichl_space2, dirichl_space1, dirichl_space1) #0
Z121 = bempp.api.ZeroBoundaryOperator(neumann_space2, dirichl_space1, dirichl_space1) #0

#Dominio de la capa intermedia Ωi en la interfaz interior
K211 = bempp.api.operators.boundary.laplace.double_layer(dirichl_space1, neumann_space1, neumann_space1, assembler=Assemble) #K
V211 = bempp.api.operators.boundary.laplace.single_layer(neumann_space1, neumann_space1, neumann_space1, assembler=Assemble) #V
K221 = bempp.api.operators.boundary.laplace.double_layer(dirichl_space2, neumann_space1, neumann_space1, assembler=Assemble) #K
V221 = bempp.api.operators.boundary.laplace.single_layer(neumann_space2, neumann_space1, neumann_space1, assembler=Assemble) #V

#Dominio de la capa intermedia Ωi en la interfaz superior
K212 = bempp.api.operators.boundary.laplace.double_layer(dirichl_space1, dirichl_space2, dirichl_space2, assembler=Assemble) #K
V212 = bempp.api.operators.boundary.laplace.single_layer(neumann_space1, dirichl_space2, dirichl_space2, assembler=Assemble) #V
K222 = bempp.api.operators.boundary.laplace.double_layer(dirichl_space2, dirichl_space2, dirichl_space2, assembler=Assemble) #K
V222 = bempp.api.operators.boundary.laplace.single_layer(neumann_space2, dirichl_space2, dirichl_space2, assembler=Assemble) #V

#Dominio del solvente Ωs
Y312 = bempp.api.ZeroBoundaryOperator(dirichl_space1, neumann_space2, neumann_space2) #0
Z312 = bempp.api.ZeroBoundaryOperator(neumann_space1, neumann_space2, neumann_space2) #0
if k==0:
    K322 = bempp.api.operators.boundary.laplace.double_layer(dirichl_space2, neumann_space2, neumann_space2, assembler=Assemble) #K
    V322 = bempp.api.operators.boundary.laplace.single_layer(neumann_space2, neumann_space2, neumann_space2, assembler=Assemble) #V
else:
    K322 = bempp.api.operators.boundary.modified_helmholtz.double_layer(dirichl_space2, neumann_space2, neumann_space2, k, assembler=Assemble) #K
    V322 = bempp.api.operators.boundary.modified_helmholtz.single_layer(neumann_space2, neumann_space2, neumann_space2, k, assembler=Assemble) #V

In [7]:
#Creacion de funcion del potencial de Coulomb  
@bempp.api.complex_callable(jit=False)
def U_c(x, n, domain_index, result):
    global Q,PC,em
    result[:] = (1 / (4.*np.pi*em))  * np.sum( Q / np.linalg.norm( x - PC, axis=1))
Uc1 = bempp.api.GridFunction(dirichl_space1, fun=U_c)  

#Construccion del vector derecho
if SF==False:
    # Rhs en Ωm
    rhs_M = (Uc1).projections(dirichl_space1) # uc
    # Rhs en Ωi en interfaz inferior
    rhs_I1 = np.zeros(neumann_space1.global_dof_count) #0
    # Rhs en Ωi en interfaz superior
    rhs_I2 = np.zeros(dirichl_space2.global_dof_count) #0
    # Rhs en Ωs
    rhs_S = np.zeros(neumann_space2.global_dof_count) #0
    # La combinacion del rhs
    rhs = np.concatenate([rhs_M, rhs_I1, rhs_I2, rhs_S])
else:
    Uc2 = bempp.api.GridFunction(dirichl_space2, fun=U_c) 
    rhs = [I1d*Uc1, 0*I1n*Uc1, 0*I2d*Uc2, 0*I2n*Uc2] 

/home/mauricioguerrero/Software/miniconda3/envs/bempp_prod/lib/python3.8/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)
/home/mauricioguerrero/Software/miniconda3/envs/bempp_prod/lib/python3.8/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


In [8]:
#Construccion matriz izquierda 4x4
if SF==False:
    blocks = [[None,None,None,None],[None,None,None,None],[None,None,None,None],[None,None,None,None]] 
    
    #Posicion de la matriz 4x4
    blocks[0][0] = (0.5*I1d+K111).weak_form()  # 0.5+K  
    blocks[0][1] = -V111.weak_form()           # -V
    blocks[0][2] = Y121.weak_form()            # 0
    blocks[0][3] = Z121.weak_form()            # 0

    blocks[1][0] = (0.5*I1n-K211).weak_form()  # 0.5-K
    blocks[1][1] = (em/ei)*V211.weak_form()    # (em/ei)V
    blocks[1][2] = K221.weak_form()            # K
    blocks[1][3] = -V221.weak_form()           # -V

    blocks[2][0] = -K212.weak_form()           # -K
    blocks[2][1] = (em/ei)*V212.weak_form()    # (em/ei)V
    blocks[2][2] = (0.5*I2d+K222).weak_form()  # 0.5+K
    blocks[2][3] = -V222.weak_form()           # -V

    blocks[3][0] = Y312.weak_form()            # 0
    blocks[3][1] = Z312.weak_form()            # 0
    blocks[3][2] = (0.5*I2n-K322).weak_form()  # 0.5-K
    blocks[3][3] = (ei/es)*V322.weak_form()    # (ei/es)V
    
    blocked = bempp.api.assembly.blocked_operator.BlockedDiscreteOperator(np.array(blocks)) 

else:
    blocks = bempp.api.BlockedOperator(4,4)  
    
    #Posicion de la matriz 4x4
    blocks[0,0] = (0.5*I1d+K111) # 0.5+K  [0][0]
    blocks[0,1] = -V111          # -V
    blocks[0,2] = Y121           # 0
    blocks[0,3] = Z121           # 0

    blocks[1,0] = (0.5*I1n-K211) # 0.5-K
    blocks[1,1] = (em/ei)*V211   # (em/ei)V
    blocks[1,2] = K221           # K
    blocks[1,3] = -V221          # -V

    blocks[2,0] = -K212          # -K
    blocks[2,1] = (em/ei)*V212   # (em/ei)V
    blocks[2,2] = (0.5*I2d+K222) # 0.5+K
    blocks[2,3] = -V222          # -V

    blocks[3,0] = Y312           # 0
    blocks[3,1] = Z312           # 0
    blocks[3,2] = (0.5*I2n-K322) # 0.5-K
    blocks[3,3] = (ei/es)*V322   # (ei/es)V

In [9]:
#Se resuelve la solucion de la Ecuacion matricial Ax=B
#Contador de iteraciones
it_count = 0
def count_iterations(x):
    global it_count
    it_count += 1
    if (it_count / 10000) == (it_count // 10000):
        print(it_count,x)

# Solucion por GMRES
from scipy.sparse.linalg import gmres
if SF==False:
    start1 = time.time()
    soln, info = gmres(blocked, rhs, M=None, callback=count_iterations,tol=1e-6)  #Sin Precondicionador#
    end1 = time.time() 
else:
    #bempp.api.enable_console_logging('info')
    start1 = time.time()
    soln, info, res, it_count = bempp.api.linalg.gmres(blocks, rhs, return_residuals=True, return_iteration_count=True, use_strong_form=True,tol=1e-6)
    end1 = time.time() 
    
# Tiempo de resolver la ecuacion
curr_time1 = (end1 - start1) 
print("Numero de iteraciones de GMRES: {0}".format(it_count))
print("Tiempo total en GMRES: {:5.2f} [s]".format(curr_time1))    

soln_u1 = soln[:dirichl_space1.global_dof_count]
soln_du1 = soln[dirichl_space1.global_dof_count : dirichl_space1.global_dof_count + neumann_space1.global_dof_count]
soln_u2 =  soln[dirichl_space1.global_dof_count + neumann_space1.global_dof_count : dirichl_space1.global_dof_count + neumann_space1.global_dof_count + dirichl_space2.global_dof_count]
soln_du2 = soln[dirichl_space1.global_dof_count + neumann_space1.global_dof_count + dirichl_space2.global_dof_count:]

/home/mauricioguerrero/Software/miniconda3/envs/bempp_prod/lib/python3.8/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


Numero de iteraciones de GMRES: 136
Tiempo total en GMRES:  3.91 [s]


In [10]:
#Calcula todo el dominio global del potencial a partir de la solucion de los borde de ambas interfaces.
if SF==False:
    # Solucion de la funcion con Dirichlet en el borde inferior
    dirichlet_fun1 = bempp.api.GridFunction(dirichl_space1, coefficients=soln_u1)
    # Solucion de la funcion con Neumann en el borde inferior
    neumann_fun1 = bempp.api.GridFunction(neumann_space1, coefficients=soln_du1)
    # Solucion de la funcion con Dirichlet en el borde superior
    dirichlet_fun2 = bempp.api.GridFunction(dirichl_space2, coefficients=soln_u2)
    # Solucion de la funcion con Neumann en el borde superior
    neumann_fun2 = bempp.api.GridFunction(neumann_space2, coefficients=soln_du2)
else:
    dirichlet_fun1 = soln[0] 
    neumann_fun1 = soln[1] 
    dirichlet_fun2 = soln[2] 
    neumann_fun2 = soln[3]  

In [11]:
#Calculo del potencial en la posicion de los atomos de la molecula
VF1 = bempp.api.operators.potential.laplace.single_layer(neumann_space1, np.transpose(PC)) 
KF1 = bempp.api.operators.potential.laplace.double_layer(dirichl_space1, np.transpose(PC))
uF = VF1*neumann_fun1 - KF1*dirichlet_fun1 

#Resultado de la energia de solvatacion por atomo
for i in range(len(PC)):
    Sum1 = (uF[0][i]).real*Q[i]
    Ei = 0.5*4.*np.pi*332.064*(Sum1)
    print(Ei)
    
#Resultado de la energia de solvatacion total    
E_Solv = 0.5*4.*np.pi*332.064*np.sum(Q*uF).real 
print('Energia de Solvatacion: {:7.6f} [kCal/mol]'.format(E_Solv) )

end = time.time()
curr_time = (end - start)   
print("Tiempo total: {:5.2f} [s]".format(curr_time))

-22.789137672964202
-22.79055139643908
-22.790112657348978
Energia de Solvatacion: -68.369802 [kCal/mol]
Tiempo total: 25.91 [s]
